In [ ]:
import fiftyone as fo
from ultralytics import YOLO

In [ ]:
# Load Voxel51 dataset
dataset_v51 = fo.load_dataset("fisheye8k-100")
classes = dataset_v51.default_classes

In [ ]:
# Dataset export in YOLO format
for split in ["train", "test","val"]:
    split_view = dataset_v51.match_tags(split)

    if split == "test":  # YOLO expects train and val
        split = "val"

    split_view.export(
        dataset_type=fo.types.YOLOv5Dataset,
        export_dir="ultralytics_data",
        label_field="detections",
        classes=classes,
        split=split,
    )

In [ ]:
# Train model
MODEL_NAME = "yolo12n"
model = YOLO(MODEL_NAME, task="detect")
results = model.train(
    data="ultralytics_data/dataset.yaml",
    project="ultralytics_runs",
    epochs=36,
    patience=3,
    batch=16,
    imgsz=512,
    exist_ok=True,
    amp=True,
)

In [ ]:
metrics = model.val()
print(f"Training finished. Evaluation: {metrics}")

In [ ]:
# Apply model to Voxel51 dataset
eval_view = dataset_v51.match_tags("test")
eval_view.apply_model(model, label_field="yolo12n", confidence_thresh=0.2)

In [ ]:
# Evaluate detections
eval_view.evaluate_detections(
    "yolo12n",
    gt_field="detections",
    eval_key="eval_yolo12n",
    compute_mAP=True,
)

In [ ]:
# Launch Voxel51 GUI
fo.launch_app(eval_view)